# Plotting FFT data

In [1]:
!which python
import sys
sys.executable

/Users/stianismar/Dropbox/gitProsjekter/master-thesis/venv/bin/python


'/Users/stianismar/Dropbox/gitProsjekter/master-thesis/venv/bin/python'

In [8]:
import numpy as np
import wt_data
import ff_transform
from matplotlib.collections import PolyCollection
import random

# For Gearbox
This feature has the name 'GbxHssRr;0,0102;m/s2'.

In [13]:
'''
wt_name is either 'WTG01', 'WTG02', 'WTG03', or 'WTG04'

'''
def load_interval(wt_name, BINS,SENSOR_NAME, load_minimal=False):
    wt_instance = wt_data.load_instance(wt_name, load_minimal=False)
    print(f"This is the amount of intervals: {len(wt_instance.ten_second_intervals)}")
    
    y = []
    x = []
    z = []
    i = 0
    for interval in wt_instance.ten_second_intervals:
        ts = interval.sensor_df['TimeStamp'].values  # Have this as the y-axis to see how the RMS/frequencies develop

        # y_val = np.array([ts[0]])

        # Have the x axis as frequency!
        # Have the Z axis as amplitude of frequency...
        try:
            vibration_signal = interval.sensor_df[SENSOR_NAME]
        except:
            continue


        y_repeated = np.repeat(i, 50)  # Repeat this y value n times to use as the y value for the corresponding x (frequency) and z (magnitude)
        y.append(y_repeated)
        i = i + 1

        comp_type = 'gearbox'
        fast = ff_transform.FastFourierTransform(vibration_signal, ts, comp_type)
        fft, time, centroid, rms, rms_bins, bin_freq = fast.fft_transform_time(calc_rms_for_bins=True,
                                                                     plot=False,
                                                                     bins=BINS,
                                                                     plot_vertical_lines=False)
        N = fast.s.size
        T = fast.t[1] - fast.t[0]
        f = np.linspace(0, 1 / T, N, )
        f = f[:N // 2]



        # z_amp = np.abs(fft)[:N // 2] * 1 / N
        z.append(rms_bins)

        x.append(bin_freq)
    x = np.array(x)
    y = np.array(y)
    z = np.array(z)
    return x,y,z

In [ ]:
SENSOR_NAME = 'GbxHssRr;0,0102;m/s2'
BINS = 50

freqs_wt01, interval_nums_wt01, rms_amplitudes_wt01 = load_interval('WTG01',
                                                                    BINS,
                                                     SENSOR_NAME,
                                                     load_minimal=False)

print(freqs_wt01.shape)
print(rms_amplitudes_wt01.shape)
print(interval_nums_wt01.shape)


**freqs_wt01, interval_nums_wt01, rms_amplitudes_wt01** can now be used to make 3d plots. The frequencies are a part of the x axis, the amplitude is the z axis and the y axis is the time 

## Creating a 3d plot using PolyCollection

In [ ]:
'''
    Create a 3d-plot using the poly collection module from matplotlib.
'''

from matplotlib import cm, pyplot as plt

def print3d_with_poly_collection(x,y,z):
    
    # Get the numpy arrays on the correct shape
    freq_data = x.T
    amp_data = z.T
    rad_data = np.linspace(0,1,amp_data.shape[1])
    
    
    verts = []
    col = []
for i in range(freq_data.shape[1]):
    a = [0, random.uniform(0, 0.1),random.uniform(0.5, 1.0)]
    col.append(a)
    
    for irad in range(len(rad_data)):
        # I'm adding a zero amplitude at the beginning and the end to get a nice
        # flat bottom on the polygons
        xs = np.concatenate([[freq_data[0,irad]], freq_data[:,irad], [freq_data[-1,irad]]])
        ys = np.concatenate([[0],amp_data[:,irad],[0]])
        verts.append(list(zip(xs, ys)))
    from matplotlib import cm


    poly = PolyCollection(verts, cmap = cm.coolwarm)
    # facecolors = col


    # facecolors = ['r', 'g', 'c', 'y','r', 'g', 'c', 'y','r', 'g', 'c']
    poly.set_alpha(0.7)
    # poly.set_cmap('blues')

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # The zdir keyword makes it plot the "z" vertex dimension (radius)
    # along the y axis. The zs keyword sets each polygon at the
    # correct radius value.
    ax.add_collection3d(poly, zs=rad_data, zdir='y')

    ax.set_xlim3d(freq_data.min(), freq_data.max())
    ax.set_xlabel('Frequency')
    ax.set_ylim3d(rad_data.min(), rad_data.max())
    ax.set_ylabel('Radius')
    ax.set_zlim3d(amp_data.min(), amp_data.max())
    ax.set_zlabel('Amplitude')

    plt.show()

## Creating surface 3d plots with axes3d

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

def print3d_with_axes3d(x,y,z):
    X = np.array(x)
    Y = np.array(y)
    Z = np.array(z)
    
    X = X.T
    Y = Y.T
    Z = Z.T
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    surf = ax.plot_surface(X, Y, Z, cmap=cm.viridis,
                           linewidth=0, antialiased=False)
    plt.title("Frequency development over intervals for WT")
    plt.show()